# Animations: MLD + SSH

This is a notebook in which we realize an animatio of the MLD+SSH countours over one of the different regions described in [Regions](./Distribution-regions.ipynb) The, using the ARMOR-3D dataset. Using the 2005-2018 period. As an example, only the plots for the Labrador and Irminger Seas region are shown. The parameters that change for each of the regions are also included as commented lines in the code. 

The code consists in a loop that plots each one of the time steps of the dataset over the regions. The plots are saved in a directory. To obtain the animation (in mp4 format) a line command in terminal has to be executed. This line is also included in this notebook, in commented lines at the final. 


This notebook has the following structure:

   [1. Imported modules](#1.-Imported-modules)
   
   [2. Dictionaries used](#2.-Dictionaries-used)
   
   [3. Functions](#3.-Functions)
   
   [4. Example plot](#4.-Example-plot)
   
The animations obtained here, can be found in the directory [Animations](/Documents/Scripts/Animations/).

## 1. Imported modules

In [1]:
import cartopy
import cartopy.feature as cfeat
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import xarray as xr
import numpy as np

import datetime
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

from matplotlib import colors
from matplotlib import ticker
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable


import os
import multiprocessing


## This module disable the printing of warnings in the notebook
import warnings; warnings.simplefilter('ignore')

## 2. Dictionaries used

In [2]:
#########################
######### DICTIONARIES DEFINITION 
#########################

"""
reg_ext is a dictionary that has the coordinate
boundaries of the main regions. The format is:
<region> :
  <lon> : (lon1, lon2)
  <lat> : (lat1,  lat2)
  <name> : title for an eventual ploting title
"""

reg_ext = {
    'lab': {
        'lon' : (-60, -30),
        'lat' : (50, 65),
        'name' : 'Labrador and Irminger Seas'
    },
    'gul': {
        'lon' : (-75, -45),
        'lat' : (30, 45),
        'name' : 'Gulf Stream'
    },
    'noe': {
        'lon' : (-30, -5),
        'lat' : (45, 60),
        'name' : 'North East Sea'
    },
    'nor':{
        'lon' : (-25, 20),
        'lat' : (65, 80),
        'name' : 'North Atlantic Ocean'
    },
    'noa':{
        'lon' : (-90, 50.),
        'lat' : (10, 85),
        'name' : 'North Atlantic Ocean'
    }
}

## 3. Functions

In [3]:
## Function to crop the dataset
def Crops(coord, data_set, dic):
    """Crops is a function that takes a data set and crops it into a 
    smaller region, using as parameters the values given by the dictionary. 
        
    Parameters:
    ------------
    coord : string
        Key value that identifies the region to obtain.

    data_set : DataArray
        Dataset to be cropped.

    dic : dictionary
        Dictionary with the boundaries of the regions.

    Output:
    -------
    new_ds : DataArray
        New data array corresponding to the region stated by 'coord'.
    """
    
    lon1, lon2 = dic[coord]['lon']
    lat1, lat2 = dic[coord]['lat']
    name_fig = dic[coord]['name']
    
    new_ds = data_set.sel(longitude=slice(lon1, lon2), latitude=slice(lat1, lat2))
    return(new_ds)


def Grid(data_set):
    """
        Grid is a function that creates a rectangular grid using as x a longitude
        array and for y a latitude array.
        
        Parameters:
        ------------
        data_set : DataArray
            Is the dataset from which we will plot the histogram.
        
        Output:
        -------
        (x, y) : n-arrays
            Arrays that correspond for each (lon,lat) point
    """
    x = data_set.longitude
    y = data_set.latitude
    
    x, y = np.meshgrid(x, y)
    return(x, y)


def Map(ax, region, xtick, ytick):
    """
        Map is a function that helps to plot the map on a certain projection
        of a region. It always uses 50m definition of coastlines. In Cartopy 
        0.16.0 it is not possible to use xticks and yticks for non-rectangular
        projections, this function is available for Cartopy 0.18.0

        Parameters:
        ------------
        ax : axes
            Current axes object in which is drawn the map.

        region : list
            Set of limit coordinates of the region to plot.
            It has the shape [lon1, lon2, lat1, lat2]

        xtick : np.array
            array containing the number and spacement of ticks 
            in x-axis.

        ytick : np.array
            Array containing the number and spacement of ticks 
            in y-axis.
    """
    ax.set_global()
    ax.coastlines('50m')
    ax.set_xticks(xtick, crs=ccrs.PlateCarree())  # Comment for the Nordic Seas
    ax.set_yticks(ytick, crs=ccrs.PlateCarree())  # Comment for the Nordic Seas
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()

    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
    ax.set_extent(region, crs=ccrs.PlateCarree())     #lat, lon
    ax.gridlines(crs=ccrs.PlateCarree(), linewidth=1, 
                 color='gray', alpha=0.5, linestyle='-.')

## 4. Example plot

In [4]:
def generate_one_figure(it=1):
    print(it)
    
    ## Opening datasets
    dir_1 = '../../../New_ARMOR/'
    dir_15 = '/net/data0-gw/data0/project/drakkar/USERS/lgarcia/data_ARMOR/'
    dir_o = '../../dummy_images/'

    fl_n1 = 'NARMOR_*.nc'
    fl_n15 = 'ARMOR_*.nc'

    c_armor1 = xr.open_mfdataset(dir_1 + fl_n1, concat_dim='time')
    c_armor15 = xr.open_mfdataset(dir_15 + fl_n15)
    
    ## Adding time stamps for the data
    d1 = datetime.datetime(2005,1,5)
    d2 = datetime.datetime(2018,12,19)
    weeks = pd.date_range(start=d1, end=d2, freq='7D')
    c_armor1.coords['time'] = (('time'), weeks)
    
    
    ## Selecting the region
    r = 'lab'
    gulf1 = Crops(r, c_armor1, reg_ext)
    gulf2 = Crops(r, c_armor15, reg_ext)
    
    del c_armor1
    del c_armor15
    
    ## min & max imits of the colorbar
    vmin = 20.
    vmax = 2500.
    
    ## Preparing the region coordidnates: ticks and grid
    lon1, lon2 = reg_ext[r]['lon']
    lat1, lat2 = reg_ext[r]['lat']
    xtick = np.linspace(lon1, lon2, num=5)
    ytick = np.linspace(lat1, lat2, num=4)
    
    x_n, y_n = Grid(gulf1)
    
    ## Choosing the region to use
    projection = ccrs.PlateCarree(360)
    region = [lon1, lon2, lat1, lat2]    #Lon, Lat
    
    ## Getting time stamp for image identification
    time = pd.to_datetime(gulf1.time.values[it])
    
    mpl.use('Agg')
    
    
    ## Start the figure
    fig, ax = plt.subplots(nrows=1, ncols=1, subplot_kw=dict(projection=projection), 
                           figsize=(10,5))   
    ## Generating the map
    Map(ax, region, xtick, ytick)
    
    ## Plot for the MLD. Color map and colorbar
    contr1 = ax.contourf(x_n, y_n, gulf1.mlotst[it], levels=np.linspace(vmin,vmax,20), 
                         cmap='plasma_r', extend='both')
    cbar = fig.colorbar(contr1, ax=ax, shrink=0.8, orientation='vertical', 
                        label='MLD [m]')  
    ## Plot of SSH contous                 
    contr2 = ax.contour(x_n, y_n, gulf2.zo[it,0], 15, linewidths=0.5, colors='k')
    
    ax.set_title(time)
    
    fig.savefig(os.path.abspath(os.path.sep.join([dir_o,"Lab_%0.4d.png" % it])))
    plt.close(fig)
    return None

In [6]:
### I do it  in two loops, because doing it in just one
### the loop never actually finishes. Uncommnet the next lines:

print('Use %i processes' % multiprocessing.cpu_count() ) 

#with multiprocessing.Pool() as pool:
#    pool.map(generate_one_figure, np.arange(0,30))     
    
#with multiprocessing.Pool() as pool:
#    pool.map(generate_one_figure, np.arange(300,729))    

Use 4 processes


In [ ]:
### To create the *.mp4 file using all the figures
### paste the next lines in the terminal


# ffmpeg -r 5 -f image2 -s 1920x1080 -i dummy_images/Lab_%04d.png -vcodec libx264 -crf 25 
# -pix_fmt yuv420p /home/lgarcia/Documents/Scripts/Animations/Gul-New.mp4 -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2"
